In [22]:
import src.Omok as omok
import numpy as np

In [23]:
def Whosstone(who):
    if ("p1" == who):
        stone = 1
    else:
        stone = 2
    return stone


In [24]:
L = 3
N = L*L
Game = omok.Omok(L)
Game.Initialize()
Tree = []

Game.Initialize()

leaf_p = [float("{0:.2f}".format(np.random.rand())) for i in range(N)]
leaf_v = float("{0:.2f}".format(np.random.rand()))
cand_player = "p1"
c_puct = 1.
tau = 1 # for first 30 moves of each game (or 0, after 30 moves)

epsilon = 0.25

In [25]:
def MCTS_SELECT (c_puct):
    ### PARAMETER SETTING ###

     # Set the First Player


    #########################
    
    global Tree
    global IDX
    global Tree_Net
    global Nt
    global Wt
    global Qt
    global Pt
    global Trajectory
    global Trajectory_A


    #########################

    #########################


    # Current Board Registration on the Tree
    current_state = Game.Board.copy()
    print (current_state)

    p1_num = current_state.count(1)
    p2_num = current_state.count(2)

    if (p2_num<p1_num):
        cand_player = "p2"
    else:
        cand_player = "p1"


    # when you run this process,
    # If the tree is empty or needs a new tree, 
    # part of "except" will be run the except
    # other case, find the state in the tree and append Trajectory

    try :
        current_idx = Tree.index(current_state)

    except:
        IDX = [0]
        Tree = [current_state]
        Tree_Net = [[]]
        Nt = [[0 for i in range(N)]]
        Wt = [[0 for i in range(N)]]
        Qt = [[0 for i in range(N)]]
        Pt = [leaf_p]
        print ("except case")
        current_idx = Tree.index(current_state)

    Trajectory = [current_state]
    Trajectory_A = []

    Diri_Pt = []
    alpha = [0.03 for i in range(len(Pt[current_idx]))]
    Diri_dist = np.random.dirichlet(alpha)
    for i in range(len(Pt[current_idx])):
        Diri_Pt.append((1.-epsilon)*Pt[current_idx][i] + epsilon*Diri_dist[i])
    
    while(1):

        ## All Tree Search Process based on the Trajectory
        current_state = Trajectory[-1]
        current_idx = Tree.index(current_state)

        if (not current_state.count(0)):
            print ("End of the Tree")
            return []

        ## Select the Stone Colour
        stone = Whosstone(cand_player)

        ## Candidate Action list for all candidate
        Cand_Action = []
        ## Regal_move list for all candidate
        Regal_move = []

        ## Cadidate Action List will be produce
        

        for i in range(N):
            if (current_state[i] == 0):
                tmp = current_state.copy()
                tmp[i] = stone
                Regal_move.append(i)
                
        # Calculate Sum of N(s,b)
        sum_N = 0
        for nn_idx in Regal_move:
            sum_N = sum_N + Nt[current_idx][nn_idx]

        # Calculate U(s,a)
        For_argmax = []
        for nn_idx in Regal_move:
            tmp_p = Pt[current_idx][nn_idx]
            tmp_q = Qt[current_idx][nn_idx]
            tmp_n = Nt[current_idx][nn_idx]
            For_argmax.append(float(c_puct) * float(tmp_p) * np.sqrt(sum_N) /float(1.+tmp_n) 
                              + float(tmp_q))

        A_t_value = max(For_argmax)
        A_t_idx = For_argmax.index(A_t_value)
        
        A_t = current_state.copy()
        A_t[Regal_move[A_t_idx]] = stone
        Trajectory_A.append(Regal_move[A_t_idx])

        try: 
            next_idx = Tree.index(A_t)
            Trajectory.append(A_t)
            if (cand_player == "p1"):
                cand_player = "p2"
            else:
                cand_player = "p1"

        except:
            print ("break")
            break

    return A_t


In [26]:
def MCTS_EXPAND(A_t):
    
    global Tree
    global IDX
    global Tree_Net
    global Nt
    global Wt
    global Qt
    global Pt
    global Trajectory
    global Trajectory_A
  
    ## EXPAND AND EVALUATE
    prev_state = Trajectory[-1]
    leaf_state = A_t

    IDX.append(len(Tree))
    Tree.append(leaf_state.copy())
    Tree_Net.append([])
    prev_idx = Tree.index(prev_state)
    Tree_Net[prev_idx].append(IDX[-1])

    leaf_p = [float("{0:.2f}".format(np.random.rand())) for i in range(N)]
    leaf_v = float("{0:.2f}".format(np.random.rand()))

    Pt.append(leaf_p.copy())
    Nt.append([0 for i in range(N)])
    Wt.append([0 for i in range(N)])
    Qt.append([0 for i in range(N)])
    
    return leaf_v

In [27]:
def MCTS_BACKUP(leaf_v,A_t):

    global Tree
    global IDX
    global Tree_Net
    global Nt
    global Wt
    global Qt
    global Pt
    global Trajectory
    global Trajectory_A

    ## BACK UP

    Trajectory.reverse()
    Trajectory_A.reverse()

    i = 0
    a_idx = -1
    for prev in Trajectory:

        idx = Tree.index(prev)
        a_idx = Trajectory_A[i]

        Nt[idx][a_idx] = Nt[idx][a_idx]+1
        Wt[idx][a_idx] = Wt[idx][a_idx]+leaf_v
        Qt[idx][a_idx] = Wt[idx][a_idx]/Nt[idx][a_idx]
        
        i = i+1
        

In [28]:
def MCTS_PLAY (tau):
    global Tree
    global IDX
    global Tree_Net
    global Nt
    global Wt
    global Qt
    global Pt
    global Trajectory
    
    ## Play

    current_state = Game.Board.copy()
    current_idx = Tree.index(current_state)

    sum_N = 0
    for idx in range(N):
        sum_N = sum_N + Nt[current_idx][idx]

    best_cand = -1
    BEST_IDX = -1
    
    Policy = []
    Proport = []
    for idx in range(N):    
        if (Nt[current_idx][idx] != 0):
            policy_value = np.power(Nt[current_idx][idx],1/tau) / np.power(sum_N,1/tau)
            Policy.append(policy_value)
        else:
            Policy.append(0)
                
    #print(current_state)
    #print(Policy)
    
    for idx in range(N):
    
        if idx == 0:
            if (current_state[idx] == 0):
                Proport.append(Policy[idx])
            else:
                Proport.append(0)
        else:
            if (current_state[idx] == 0):
                Proport.append(Policy[idx]+Proport[idx-1])
            else:
                Proport.append(Proport[idx-1])
    
    print(Proport)
    
    selector = np.random.rand()
    #print (selector)
    cand_action = -1
    
    for idx in range(N):
        if (selector < Proport[idx]):
            cand_action = idx
            break
    print (cand_action)

    #####
    ## ADD the probability of prportionally 
    #####
    
    now_player = ""
    x = -1
    y = -1

    np1 = current_state.count(1)
    np2 = current_state.count(2)
    
    if np1 <= np2:
        now_player = "p1"
    else:
        now_player = "p2"
        
    x = cand_action % L
    y = cand_action // L
    
    return now_player, x, y
    
    ## Add Resign Rate

In [30]:
for j in range(9):
    for i in range(3):
        A_t = MCTS_SELECT(c_puct)
        print (A_t)
        if (A_t):
            leaf_v = MCTS_EXPAND(A_t)
            MCTS_BACKUP(leaf_v,A_t)
        else:
            break

    print ("Tree")
    print (Tree)
    print ("Tree_Net")
    print (Tree_Net)
    print ("Nt")
    print (Nt)
    print ("Qt")
    print (Qt)
    print ("Wt")
    print (Wt)
    print ("Pt")    
    print (Pt)
    print ("Trajectory")
    print (Trajectory)
    print ("Trajectory_A")
    print (Trajectory_A)

    plyer, x, y= MCTS_PLAY(tau)
    Game.Playing(plyer,x,y)
    print(Game.PrintBoard())
    input()

[2, 1, 2, 1, 0, 2, 1, 1, 0]
End of the Tree
0
Tree
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [2, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 2, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 2], [2, 1, 0, 0, 0, 0, 0, 1, 0], [2, 0, 0, 0, 0, 0, 1, 1, 0], [2, 2, 0, 0, 0, 0, 1, 1, 0], [2, 0, 0, 0, 0, 2, 1, 1, 0], [2, 0, 2, 0, 0, 0, 1, 1, 0], [2, 0, 0, 2, 0, 0, 1, 1, 0], [2, 1, 0, 0, 0, 2, 1, 1, 0], [2, 1, 2, 0, 0, 2, 1, 1, 0], [2, 0, 0, 1, 0, 2, 1, 1, 0], [2, 1, 2, 1, 0, 2, 1, 1, 0], [2, 1, 0, 2, 0, 2, 1, 1, 0], [2, 1, 1, 2, 0, 2, 1, 1, 0], [2, 1, 2, 0, 1, 2, 1, 1, 0], [2, 1, 2, 2, 1, 2, 1, 1, 0], [2, 1, 2, 2, 1, 2, 1, 1, 1], [2, 1, 2, 1, 2, 2, 1, 1, 0], [2, 1, 2, 1, 2, 2, 1, 1, 1]]
Tree_Net
[[1, 2, 3], [], [], [4, 5, 6], [7, 8], [], [], [], [9, 10, 11, 12], [], [13, 15], [], [], [14, 17], [16, 19], [], [22], [18], [], [20], [21], [], [23], []]
Nt
[[1, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 

In [21]:
MCTS_PLAY(tau)
print(Game.PrintBoard())

[0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
2
O O - 
X - X 
- - O 
None
